In [ ]:
%matplotlib inline

In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import lmfit as lf

from scipy.stats import norm

# functions written for this project
import utils
import two_stage_model as model

### Orientation Suppression

In [ ]:
os_df = utils.load_all_data("cleanData/*filteredData_OS.mat", utils.load_individual_os_data)

In [ ]:
# Since -1 and -2 represent baseline, these should both be true
print(np.all(os_df[os_df.Orientation==-1]['MaskContrast']==0))
print(np.all(os_df[os_df.Orientation==-2]['MaskContrast']==0))

In [ ]:
os_df.head(n=5)

In [ ]:
gvars = ["Subject", "Eye", "Orientation", "Presentation"] # Presentation conditions
gvars_mask = gvars + ["MaskContrast"] # Mask contrast, m in the model
gvars_masktarget = gvars_mask + ["ProbeContrastUsed"] # Target contrast, t in the model

In [ ]:
grouped, condensed_df = utils.summarize_conditions(os_df, gvars_masktarget)

In [ ]:
df = condensed_df.reset_index()
df.head(n=10)

#### Develop a function that should be minimized to fit the model

Within each staircase, you have numerous trials at different MaskContrasts, and as outcomes you have ResponseAccuracy (0|1). In the paper they describe these as the number of trials (n) and correct responses (c) for each target (here, ProbeContrastUsed) and mask (MaskContrast) level.

Ultimately, they fit their parameters (p, q, z, sigma_int - but mine will be different since i'm using a different model) by minimizing a likelihood function of these parameters given (vectors) m, t, n, c which are the different mask/target conditions.

Questions:
 * what are the parameters of the two-stage model, and which of those are going to be fixed and which are going to be free to vary, ie fit?
 * What is this normal integral d' to percent correct (P(c)) transformation?
     * Phi(t) is the cdf of the normal distribution, tells you the probability that the normal variable is less than t
 * What is the proper equation for the likelihood of the parameters given m, t, n, and c?
 * What is the proper function for calculating percent correct from the model parameters?

In [ ]:
params = lf.Parameters()
params.add('m', value=1.3, vary=False)
params.add('S', value=1, vary=False)
params.add('w_m', value=1, min=0.0, vary=True)
params.add('w_d', value=1, min=0.0, vary=True)
params.add('a', value=0, min=0.0, vary=True)
params.add('k', value=0.2, vary=False)
params.add('p', value=8, vary=False)
params.add('q', value=6.5, vary=False)
params.add('Z', value=.0085, vary=False)

In [ ]:
fitted_params = df.groupby(gvars).apply(utils.model_condition, model.two_stage_response, params)

In [ ]:
fitted_df = fitted_params.reset_index()

In [ ]:
fitted_df

In [ ]:
plt.scatter(fitted_df['w_m'], fitted_df['w_d'])

In [ ]:
sns.violinplot(fitted_df['w_m'])

In [ ]:
sns.violinplot(fitted_df['w_d'])

In [ ]:
sns.violinplot(fitted_df['a'])

### Surround Suppression

In [ ]:
ss_df = utils.load_all_data("cleanData/*filteredData_SS.mat", utils.load_individual_ss_data)

Testing LL

In [ ]:
model.loglikelihood(10, 4, .5)

In [ ]:
!pytest test_utils.py